In [1]:
%pip install transformers
%pip install PyPDF2

     |████████████████████████████████| 481kB 2.7MB/s 
     |████████████████████████████████| 3.1MB 11.3MB/s 
     |████████████████████████████████| 1.0MB 27.1MB/s 
     |████████████████████████████████| 870kB 39.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=0b6524e1c4d8b744c157ca142437d0d292ce2d31e5aa7200073d57b10364c95d
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 81kB 2.6MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=27a989e15d9772788d561ec1ec2d74fd977241a59a4833bdfb418a186688d72c
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
%rm -rf BERT_financialReportAnalysis/

In [3]:
!git clone https://github.com/francisliu2/BERT_financialReportAnalysis.git

Cloning into 'BERT_financialReportAnalysis'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 77 (delta 46), reused 50 (delta 19), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [4]:
%cd BERT_financialReportAnalysis/

/content/BERT_financialReportAnalysis


In [5]:
from companytoVector_BERT import *
from corporate_profile_extrator import *

from google.colab import files
import os
import glob

os.mkdir('outputs')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



Max number of tokens in one sentence: 512


In [0]:
CP = glob.glob("./data/*.pdf")

In [7]:
model = load_model()

In [0]:
success = []
failed = []
result_temp = [] # for debugging

for i in range(len(CP)):
  name = CP[i].replace('.pdf', '') 
  name = name.replace('./data/', '')
  corporate_profile_text = get_corporate_profile(CP[i])
  corporate_profile_ids = text_tokening(corporate_profile_text, verbose=1)
  if corporate_profile_ids == None:
    failed.append({name, 'sentence is too long'})
  else:
    success.append(name)
    result = company2vector(corporate_profile_ids, model)
    result_temp.append(result)
  np.save('./outputs/'+name, result)

In [126]:
len(corporate_profile_ids)

10

In [17]:
tokenizer.convert_ids_to_tokens(corporate_profile_ids[0][0])

'[CLS]'

In [0]:
sent = ['The startswith() method gm dddreturns True', 'another sentence']
a = []
for s in sent:
  a.append(tokenizer.encode(s,pad_to_max_length=20))


In [0]:
a

In [0]:
input2model = torch.tensor(a)
outputs = model(input2model)

# r = []
# i=-1
# for t in b:
#   if t.startswith("##"):
#     r.append(i)
#   else:
#     i+=1
#     r.append(i)

# s=0
# result  = outputs[0][s:s+1,0,:] # Start sentence starting token
# for R in np.unique(r[1:]):
#   mask = np.array(r) == R
#   temp = torch.mean(outputs[0][s:s+1,mask,:], dim=1) # Taking mean over subwords of the same word
#   result = torch.cat((result, temp))

In [73]:
r

[0, 1, 2, 2, 3, 4, 5, 6, 7, 7, 7, 7, 8, 9]

In [124]:
outputs[0].size()

torch.Size([2, 512, 768])

In [110]:
result.size()

torch.Size([10, 768])

In [111]:
outputs[0].size()

torch.Size([1, 14, 768])

In [9]:
success

['Yongnam Annual Report 2010', 'China Star_Sustainability Report']

In [0]:
!zip -r ./outputs.zip ./outputs
!sleep 2
files.download('outputs.zip')

updating: outputs/ (stored 0%)
updating: outputs/China Star_Sustainability Report.npy (deflated 7%)
updating: outputs/Yongnam Annual Report 2010.npy (deflated 7%)


In [0]:
# Check time needed to run
corporate_profile_text = get_corporate_profile(CP[0])
%time result = company2vector(corporate_profile_text, model)

CPU times: user 29.9 s, sys: 2.69 s, total: 32.6 s
Wall time: 32.7 s
